In [1]:
import numpy as np, pandas as pd, import_ipynb, re

In [2]:
file = pd.ExcelFile( 'HydrolysisRateConstants_011521.xlsx' )

In [3]:
# retrieving key sheet
key = file.parse( 'Key' )

In [4]:
# cleaning the key sheet
key.columns = key.loc[ 6, : ].fillna( 'drop' ).values
key = key.drop( labels = [ 0, 1, 2, 3, 4, 5, 6 ], axis = 0 )
key = key.drop( labels = 'drop', axis = 1 ).fillna( '' ).reset_index( drop = True )
# adding units to headers
key[ 'Column Heading' ] = ( key[ 'Column Heading' ] + ' ' + key[ 'Units' ] ).str.strip()
key = key.drop( labels = 'Units', axis = 1 )

In [5]:
# storing all other sheets except the key sheet
sheet = {}
for sheet_name in file.sheet_names[ 1 : ] :
    sheet[ sheet_name ] = pd.read_excel( 
        'HydrolysisRateConstants_011521.xlsx', 
        sheet_name = sheet_name, 
        header = None,
        dtype = str
        )
del file

In [6]:
# cleaning the dictionary
for sheet_name in sheet.copy() :
    # applying appropriate headers
    sheet[ sheet_name ].columns = key[ 'Column Heading' ].values
    # replacing sheet name with caption
    caption = re.sub( 
        pattern = ' \(.*', 
        repl = '', 
        string = sheet[ sheet_name ].iloc[ 0, 0 ] 
        )
    sheet[ caption ] = sheet[ sheet_name ].fillna( '' )
    # creating column with caption type
    sheet[ caption ][ 'reaction'] = caption
    del sheet[ sheet_name ]

In [7]:
list( sheet.keys() )

['Halogenated Aliphatics: Nucleophilic Substitution',
 'Halogenated Aliphatics: Elimination',
 'Epoxide Hydrolysis',
 'Organophosphorus Ester Hydrolysis',
 'Acid Halide Hydrolysis',
 'Acid Halide Hydrolysis: another mechanism',
 'Carboxylic Acid Ester Hydrolysis',
 'Anhydride Hydrolysis',
 'Amide Hydrolysis',
 'Carbamate Hydrolysis',
 'Urea Hydrolysis',
 'Sulfonylurea Hydrolysis',
 'Sulfonylurea Hydrolysis: Ring Contraction',
 'Carbonate Hydrolysis',
 'Lactone Hydrolysis',
 'Imide Hydrolysis',
 'Nitrile Hydrolysis',
 'Lactam Hydrolysis',
 'N-S Cleavage']

In [24]:
def is_int( x ) :
    try : 
        int( x )
        return True
    except :
        return False

In [26]:
# retrieving parent molecule names and SMILES
molecules = pd.DataFrame()
for caption, data in sheet.items() :
    # remove empty rows
    data = data[ ( data != '' ).any( axis = 1 ) ]
    # each molecule is numbered in the Example Number column
    idx = data[ data.loc[ :, 'Example Number' ].apply( is_int ) ].index.tolist()
    molecules = pd.concat( [ molecules, data.loc[ idx, [ 'Name', 'SMILES' ] ] ] )
# drop duplicated rows
molecules = molecules[ ~ molecules.duplicated( keep = 'first' ) ]

In [27]:
# retrieving product molecules and SMILES
df = pd.DataFrame()
for caption, data in sheet.items() :
    # remove empty rows
    data = data[ ( data != '' ).any( axis = 1 ) ]
    # data before the first example is redundant
    idx = data[ data.loc[ :, 'Example Number' ].apply( is_int ) ].index.tolist()
    data = data.loc[ idx[ 0 ] : ]
    df = pd.concat( 
            [ 
            df, 
            data[ 
                [ 'Name' ] +\
                [ col for col in data.columns 
                 if 'product' in col.lower() ] +\
                [ 'Notes', 'reaction' ]
                ]
            ] 
        )
# remove empty rows between the name and notes column
df = df[ ( df.iloc[ :, 1 : -2 ] != '' ).any( axis = 1 ) ]
df = df.reset_index( drop = True )

In [28]:
# duplicating parent name down rows in which multiple products are listed
idx = df[ df[ 'Name' ] != '' ].index.tolist()
for i in range( len( idx ) ) :
    if i + 1 > len( idx ) - 1 :
        df.loc[ idx[ i ] :, 'Name' ] = df.loc[ idx[ i ], 'Name' ]
        break
    df.loc[ idx[ i ] : idx[ i + 1 ] - 1, 'Name' ] = df.loc[ idx[ i ], 'Name' ]

In [29]:
# combining all products, product smiles and reaction types
data = []
for name in sorted( df[ 'Name' ].unique() ) :
    for row in df.loc[ 
        df[ 'Name' ] == name,
        [ 'Acidic Products'   , 'Acidic Product Smiles' , 'Notes about acidic product distribution' ,
          'Neutral Products'  , 'Neutral Product Smiles', 'Notes about neutral product distribution',
          'Basic Products'    , 'Basic Product SMILES'  , 'Notes about basic product distribution',
          'Overall Product(s)', 'Product(s) SMILES'     , 'Notes',
          'reaction' ]
        ].apply(
            lambda series :
                [ series[ i : i + 3 ].tolist() + [ series[ 'reaction' ] ]
                  for i in range( 0, 12, 3 ) ],
            axis = 1
            ) :
        for product, smiles, notes, reaction in row :
            data.append( [ name, product, smiles, reaction, notes ] )
products = pd.DataFrame( data, columns =  [ 'Name', 'Product', 'SMILES', 'reaction', 'Comments' ] )
# drop duplicated rows
products = products[ ~ products.duplicated( keep = 'first' ) ]
del df # don't need it anymore

In [30]:
# collecting unique molecules and smiles in products
more_molecules = products[ [ 'Product', 'SMILES' ] ]
more_molecules = more_molecules[ ~ more_molecules.duplicated( keep = 'first' ) ]
more_molecules = more_molecules.rename( columns = { 'Product' : 'Name' } )
# remove empty product names
more_molecules = more_molecules[ ( more_molecules[ 'Name' ] != '' ) ]
# remove erroneous names
more_molecules = more_molecules[ ~(
    ( more_molecules[ 'Name' ] == 'products not analyzed; but reported as those of sulfonyl urea hydrolysis' ) |
    ( more_molecules[ 'Name' ] == 'degradation products not reported' ) |
    ( more_molecules[ 'Name' ] == 'no product distribution reported' )
    ) ]
more_molecules = more_molecules.reset_index( drop = True )
# remove duplicates with empty smiles
dupes = more_molecules[ more_molecules[ 'Name' ].duplicated( keep = False ) ]
more_molecules = more_molecules.drop( labels = dupes[ dupes[ 'SMILES' ] == '' ].index.tolist(), axis = 0 )
# remaining duplicates are possibly different molecules with the same name
# Not sure how to handle these ...
del dupes # keeping them in the more molecules dataframe

In [33]:
# combine molecules with products
current_names = molecules[ 'Name' ].unique()
for name in more_molecules[ 'Name' ] :
    if name not in current_names :
        molecules = pd.concat( 
            [ molecules, more_molecules[  more_molecules[ 'Name' ] == name ] ]
             )
# remove duplicates
molecules = molecules[ ~ molecules.duplicated( keep = 'first' ) ]
molecules = molecules.reset_index( drop = True )
# create primary key
molecules = molecules.reset_index().rename( 
    columns = { 'index' : 'id', 'Name' : 'name', 'SMILES' : 'smiles' } 
    )

In [34]:
molecules

,id,name,smiles
0,0,allyl chloride,C=CCCl
1,1,chloromethane,CCl
2,2,chloroethane,CCCl
3,3,2-chloropropane,CC(C)Cl
4,4,2-Chloro-2-methylpropane,CC(C)(C)Cl
...,...,...,...
306,306,fenpropathrin acid,OC(C#N)C1=CC(OC2=CC=CC=C2)=CC=C1
307,307,"1,1'-(2,2-Dichloro-1,1-ethenediyl)bis(4-methox...",COC1=CC=C(C=C1)C(=C(Cl)Cl)C2=CC=C(C=C2)OC
308,308,Oxime,CSC(=NO)C(=O)N(C)C |w:3.3|
309,309,Phthalamic acid,C1=CC=C(C(=C1)C(=O)N)C(=O)O


In [35]:
# turning parent and child names in product table to foreign keys
# that reference the molecule table
for name, product in products[ [ 'Name', 'Product' ] ].to_numpy() :
    parent_id = molecules.loc[ molecules[ 'name' ] == name, 'id' ].iloc[ 0 ]
    child_id = molecules.loc[ molecules[ 'name' ] == product, 'id' ]
    products.loc[ products[ 'Name' ] == name, 'Name' ] = parent_id
    if child_id.size : # some parents don't have products listed
        products.loc[ products[ 'Product' ] == product, 'Product' ] = child_id.iloc[ 0 ]
    else :
        products.loc[ products[ 'Product' ] == product, 'Product' ] = ''
products = products.drop( labels = [ 'SMILES', 'reaction' ], axis = 1 )
products = products.rename( 
    columns = { 
        'Name' : 'parent_id', 'Product' : 'child_id', 'Comments' : 'comments'
        } 
    )
products.reset_index( drop = True )
products = products.reset_index().rename( columns = { 'index' : 'id' } )

In [36]:
products

,id,parent_id,child_id,comments
0,0,150,,
1,1,150,,
2,4,150,193,sulfonyl urea hydrolysis products; no ester hy...
3,6,150,193,sulfonyl urea hydrolysis products
4,8,150,195,sulfonyl urea hydrolysis products; no ester hy...
...,...,...,...,...
247,548,45,,
248,551,45,309,
249,555,45,310,
250,556,151,,


In [37]:
# extracting all other data
other = pd.DataFrame()
for name in sheet :
    df = sheet[ name ]
    # remove empty rows
    df = df[ ( df != '' ).any( axis = 1 ) ]
    # data before the first example is redundant
    idx = df[ df.loc[ :, 'Example Number' ].apply( is_int ) ].index.tolist()
    df = df.loc[ idx[ 0 ] : ]
    other = pd.concat( 
        [ 
            other, 
            df[ [ 
                'Name', 'Citation', 'Comments', 'Measurement Temperature (deg C)',
                'Ea (kcal/mol)', 'Acidic pH', 'Acidic Half-life (days)',
                'Acidic Degradation Rate (k_1)', 'Acidic Rate Constant Units',
                'Neutral pH', 'Neutral Half-life (days)', 'Neutral Degradation Rate',
                'Neutral Rate Constant Units', 'Basic pH', 'Basic Half-life  (days)', 
                'Basic Degradation rate (k_1)', 'Basic Rate Constant Units', 'reaction'
                ] ]
            ] 
        )
other = other.reset_index( drop = True )

In [38]:
# duplicating parent name down rows in which multiple products are listed
idx = other[ other[ 'Name' ] != '' ].index.tolist()
for i in range( len( idx ) ) :
    if i + 1 > len( idx ) - 1 :
        other.loc[ idx[ i ] :, 'Name' ] = other.loc[ idx[ i ], 'Name' ]
        break
    other.loc[ idx[ i ] : idx[ i + 1 ] - 1, 'Name' ] = other.loc[ idx[ i ], 'Name' ]

In [39]:
# duplicating citations down rows in which multiple measurements are listed
idx = other[ other[ 'Citation' ] != '' ].index.tolist()
for i in range( len( idx ) ) :
    if i + 1 > len( idx ) - 1 :
        other.loc[ idx[ i ] :, 'Citation' ] = other.loc[ idx[ i ], 'Citation' ]
        break
    other.loc[ idx[ i ] : idx[ i + 1 ] - 1, 'Citation' ] = other.loc[ idx[ i ], 'Citation' ]

In [40]:
# extracting all measurements
data = []
for name in sorted( other[ 'Name' ].unique() ) :
    for row in other.loc[ 
        other[ 'Name' ] == name, [ 
                'Citation', 'Comments', 'Measurement Temperature (deg C)',
                'Ea (kcal/mol)', 'Acidic pH', 'Acidic Half-life (days)',
                'Acidic Degradation Rate (k_1)', 'Acidic Rate Constant Units',
                'Neutral pH', 'Neutral Half-life (days)', 'Neutral Degradation Rate',
                'Neutral Rate Constant Units', 'Basic pH', 'Basic Half-life  (days)', 
                'Basic Degradation rate (k_1)', 'Basic Rate Constant Units', 'reaction'
                ] 
        ].apply(
            lambda series :
                [ series[ i : i + 4 ].tolist() +\
                  series[ 
                     [ 'reaction', 'Measurement Temperature (deg C)', 
                       'Ea (kcal/mol)', 'Citation', 'Comments' ] 
                     ].tolist()
                 for i in range( 4, 16, 4 ) ],
            axis = 1
            ) :
        for pH, halflife, rate, units, reaction, temp, ea, citation, comment in row :
            if pH or halflife or rate :
                data.append( 
                    [ name, pH, halflife, rate, units, reaction, temp, ea, citation, comment ] 
                )

In [41]:
measures = pd.DataFrame( data, columns = [
        'name', 'pH', 'Half-life (days)', 'Degradation Rate (k_1)', 
        'Rate Constant Units', 'reaction', 'Measurement Temperature (deg C)', 
        'Ea (kcal/mol)', 'Citation', 'Comments' 
        ] 
    )
measures = measures[ ~ measures.duplicated( keep = 'first' ) ]

In [42]:
def dtype_exception( x, dtype ) :
    if x != '' :
        try :
            dtype( x )
            return False
        except :
            return True
    return False

In [43]:
# turning parent names in product table to foreign keys
# that reference the molecule table
for name in measures[ 'name' ].unique() :
    parent_id = molecules.loc[ molecules[ 'name' ] == name, 'id' ].iloc[ 0 ]
    measures.loc[ measures[ 'name' ] == name, 'name' ] = parent_id
measures = measures.rename( 
    columns = { 
        'name' : 'parent_id', 'Half-life (days)' : 'half-life',
        'Degradation Rate (k_1)' : 'rate (k_1)', 'Rate Constant Units' : 'rate units',
        'Measurement Temperature (deg C)' : 'temperature (deg C)', 
        'Ea (kcal/mol)' : 'activation (kcal/mol)', 'Citation' : 'citation',
        'Comments' : 'comments'
        } 
    )
measures = measures.reset_index( drop = True )
measures = measures.reset_index().rename( columns = { 'index' : 'id' } )
# dealing with ranges
measures[ 'pH min' ] = ''
measures[ 'pH max' ] = ''
measures[ 'half-life min' ] = ''
measures[ 'half-life max' ] = ''
measures[ 'half-life units' ] = ''
measures[ 'rate (k_1) min' ] = ''
measures[ 'rate (k_1) max' ] = ''
measures = measures[ 
    ['id', 'parent_id', 'pH', 'pH min', 'pH max', 
     'half-life', 'half-life min', 'half-life max', 'half-life units',
     'rate (k_1)', 'rate (k_1) min', 'rate (k_1) max', 'rate units',
     'reaction', 'temperature (deg C)', 'activation (kcal/mol)', 'citation',
     'comments' ]
    ]
# remove ' ' entries
measures = measures.apply( lambda s : s.apply( lambda x : '' if x == ' ' else x ), axis = 1 )
# removing '--' entries
measures.loc[ measures[ 'half-life' ] == '--', 'half-life' ] = ''
# correcting entries with hr units
measures.loc[ measures[ 'half-life' ].apply( lambda x : 'hr' in str( x ) ), 'half-life units' ] = 'hours'
measures.loc[ 
    measures[ 'half-life' ].apply( lambda x : 'hr' in str( x ) ), 'half-life' 
    ] = measures.loc[ 
        measures[ 'half-life' ].apply( lambda x : 'hr' in str( x ) ), 'half-life' 
        ].str.strip( ' hr' )
# correcting '50,000 yr'
measures.loc[ 328, 'half-life' ] = 50000
measures.loc[ 328, 'half-life units' ] = 'years'
# correcting pH range
for indx, row in measures.loc[ measures[ 'pH' ].apply( lambda x : '-' in str( x ) ), 'pH' ].items() :
    ph_min, ph_max = row.split( '-' )
    measures.loc[ indx, 'pH' ] = ''
    measures.loc[ indx, 'pH min' ] = ph_min
    measures.loc[ indx, 'pH max' ] = ph_max
# correcting 'NR' and 'N.R. ' values
measures.loc[ measures[ 'pH' ].apply( dtype_exception, args = [ float ] ), 'pH' ] = ''
# correcting less than half-life values
for indx, row in measures.loc[ measures[ 'half-life' ].apply( lambda x : '<' in str( x ) ), 'half-life' ].items() :
    measures.loc[ indx, 'half-life max' ] = row.replace( '<', '' ).strip()
    measures.loc[ indx, 'half-life' ] = ''
# correcting greater than half-life values
for indx, row in measures.loc[ measures[ 'half-life' ].apply( lambda x : '>' in str( x ) ), 'half-life' ].items() :
    measures.loc[ indx, 'half-life min' ] = row.replace( '>', '' ).replace( 'd', '' ).strip()
    measures.loc[ indx, 'half-life' ] = ''
# correcting greater than rate values
for indx, row in measures.loc[ measures[ 'rate (k_1)' ].apply( lambda x : '>' in str( x ) ), 'rate (k_1)' ].items() :
    measures.loc[ indx, 'rate (k_1) min' ] = row.replace( '>', '' ).strip()
    measures.loc[ indx, 'rate (k_1)' ] = ''
# correcting values including kob units
for indx, row in measures.loc[ measures[ 'rate (k_1)' ].apply( lambda x : 'kobs' in str( x ) ), 'rate (k_1)' ].items() :
    ten, exp = map( int, row.replace( 'kobs', '' ).strip().split( '^' ) )
    measures.loc[ indx, 'rate (k_1)' ] = ten**exp
    measures.loc[ indx, 'rate units' ] = 'kobs'
# removing 'too fast' entries  
measures.loc[ measures[ 'rate (k_1)' ] == 'too fast', 'rate (k_1)' ] = ''
# removing other string values in rate field
measures.loc[ 
    measures[ 'half-life' ].apply( 
        lambda x : 
        'stable' in str( x ).lower() or 'very fast' in str( x ).lower() or 
        'no hydrolysis' in str( x ).lower() or 'no degradation' in str( x ).lower()
        ),
    'half-life'
    ] = ''
measures.loc[ 
    ( measures[ 'half-life' ] != '' ) & ( measures[ 'half-life units' ] == '' ),
    'half-life units'
    ] = 'days'